Sayama Exercise 18.5

#### Note: 
pycxsimulator will not work properly in a notebook (at least in the way it is implemented here) run code as a .py file. 

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
from pylab import *
import networkx as nx
import random as rd


def initialize():
    global g
    g = nx.erdos_renyi_graph(100, 0.1)
    g.pos = nx.spring_layout(g)
    for i in g.nodes:
        g.nodes[i]['state'] = 1 if random() < .5 else 0

def observe():
    global g
    cla()
    nx.draw(g, vmin = 0, vmax = 1,
            node_color = [g.nodes[i]['state'] for i in g.nodes],
            pos = g.pos)

def async_update():
    global g
    a = rd.choice(list(g.nodes))
    if g.nodes[a]['state'] == 0: # if susceptible
        b = rd.choice(list(g.neighbors(a)))
        if g.nodes[b]['state'] == 1: # if neighbor b is infected
            g.nodes[a]['state'] = 1 if random() < p_i else 0
    else: # if infected
        g.nodes[a]['state'] = 0 if random() < p_r else 1


def sync_update():
    global g
    for node in list(g.nodes):
        print "this"
        if g.nodes[node]['state'] == 0: # if susceptible
            g.nodes[node]['next_state'] = 0
            for neighbor in list(g.neighbors(node)):
                if g.nodes[neighbor]['state'] == 1: # if neighbor  is infected
                    if random() < p_i:
                        g.nodes[node]['next_state'] = 1
                        #break
        else: g.nodes[node]['next_state'] = 0 if random() < p_r else 1
    #update all
    for node in list(g.nodes):
        g.nodes[node]['state'] = g.nodes[node]['next_state']


p_i, p_r = (0.5, 0.5)  


        
import pycxsimulator
pycxsimulator.GUI().start(func=[initialize, observe, sync_update])

#### b. Question: Why does using synchronous or asynchronous updating make adifference?
In asynchronous updates, a node that would be infected by a neighbor at some time t would remain healthy if we happen to choose the infected neighbor in time t-1 and it recovers. This would not happen in synchronous updates, in which the infected neighbor can recover in time t but at the same time infect it's neighboor. Hence, whehther we do synchronous or asynchronous updates can influence the outcome.


#### c. Question: For the mean field approximation described in Section 18.5, why is itappropriate to use the synchronous update method and not the asynchronous one?

In mean field approximation we are calculating the probability that a healthy node will not be infected next, and this expression involves (1-p_eqp_i)^(n-1), which means that for every neighboor we check wether there is an edge, wether the neighbor is infected and whether the disease will be transmitted. Hence, the probability of being infected at the next step is assumed to be dependent on all the neighbors at the same time (rather than choosing a random neighboor and updating based on that). Note that synchronous updates is also how we updated CA for mean field approximation.

Experimentally confirm the “your friends have more friends than you do” observation
from pp421–422 in the textbook.

a. Generate a random network with 1000 nodes and (approximately) 20,000 edges,
so that the average degree of a node is 40. Do this for each of the following types
of random networks and compare your observations: Erdős–Renyi,
Watts–Strogatz, Barabási–Albert random graphs.

b. Write code to compute the average degree (which should be 40 — this is just to
confirm that you generated the graph correctly).

c. Write code to compute the average degree of each neighbor in the graph. To loop
through all neighbors in the graph, loop through all edges and then through each
node attached to an edge.



In [55]:
n = 1000
k = 40

#Erdos-Renyi
# p = 20,000 / (1000*999/2) = .04
g = nx.erdos_renyi_graph(1000, 20000.0/(500*999))
print(np.mean(g.degree, axis=0)[1])
print(np.mean([(g.degree[edge[0]], g.degree[edge[1]]) for edge in g.edges]))

39.576
40.5846977966


In [57]:
#Watts-Strogatz
g = nx.generators.random_graphs.watts_strogatz_graph(1000, 40, .2)
print(np.mean(g.degree, axis=0)[1])
print(np.mean([(g.degree[edge[0]], g.degree[edge[1]]) for edge in g.edges]))

40.0
40.1672


In [64]:
#Barabasi-Albert   <k> = 2m
g = nx.generators.random_graphs.barabasi_albert_graph(1000, 20)
print(np.mean(g.degree, axis=0)[1])
print(np.mean([(g.degree[edge[0]], g.degree[edge[1]]) for edge in g.edges]))

39.2
61.5350510204


##### d. Question: How does the average degree of neighbors (the number of friends of your friends) compare to the average degree of the graph (your number of friends)?

The average degree of neighbors is always larger than the average degree of the graph, which confirms the friendship paradox. This difference is small in Erdos-Renyi and Watts-Strogartz but quite large in Barabasi-Albert. This makes intuitive sense because the former have more uniform degree distributions, while the later has a scale free distribution so that sampling bias can have a notorious effect.